# Install and imports required libraries

In [2]:
! pip install --upgrade pip
! pip install scikit-learn 
! pip install lightgbm


In [3]:
! pip install --no-cache-dir --force-reinstall ../../dist/lohrasb-4.1.0-py3-none-any.whl  

Processing /Users/hjavedani/Documents/Lohrasb/dist/lohrasb-4.1.0-py3-none-any.whl
     |████████████████████████████████| 8.8 MB 319 kB/s            
  Preparing metadata (setup.py) ... done
     |████████████████████████████████| 10.2 MB 10.1 MB/s            
     |████████████████████████████████| 50 kB 15.3 MB/s            
     |████████████████████████████████| 4.4 MB 10.8 MB/s            
     |████████████████████████████████| 68 kB 9.2 MB/s             
     |████████████████████████████████| 143 kB 11.7 MB/s            
     |████████████████████████████████| 210 kB 11.3 MB/s            
     |████████████████████████████████| 103 kB 12.7 MB/s            
     |████████████████████████████████| 112 kB 11.3 MB/s            
     |████████████████████████████████| 468 kB 9.7 MB/s            
     |████████████████████████████████| 158 kB 10.3 MB/s            
     |████████████████████████████████| 189 kB 8.3 MB/s            
     |████████████████████████████████| 61 kB 3.8 MB/

In [4]:
! python3 --version

Python 3.6.15


In [5]:
import lightgbm
import lohrasb
print(lightgbm.__version__)
print(lohrasb.__version__)

[Errno 2] No such file or directory: '/Users/hjavedani/Documents/Lohrasb/prod_env/lib/python3.6/site-packages/lohrasb/config.yaml'
In this module, the default logging will be applied. The error is [Errno 2] No such file or directory: '/Users/hjavedani/Documents/Lohrasb/prod_env/lib/python3.6/site-packages/lohrasb/config.yaml' which will be skipped!
default logger setting is applied !
4.0.0
4.1.0


In [6]:
# Import necessary libraries
from lightgbm import LGBMClassifier
from sklearn.datasets import make_classification, make_regression
from sklearn.metrics import f1_score, r2_score
from sklearn.model_selection import KFold, train_test_split
from sklearn.neural_network import MLPRegressor
from lohrasb.best_estimator import BaseModel

# Define hyperparameters for the MLPRegressor and LGBMClassifier
# These will be the values that the hyperparameter search function will iterate through.
mlp_params_reg = {
    "hidden_layer_sizes": [(5, 5, 5), (5, 10, 5), (10,)],
    "activation": ["tanh", "relu"],
    "solver": ["sgd", "adam"],
    "alpha": [0.0001, 0.05],
    "learning_rate": ["constant", "adaptive"],
}
lgbm_params = {"max_depth": [5, 6, 7, 10]}


# Function for training and evaluating a classification model
def using_tune_classification(estimator, params):
    # Create a synthetic classification dataset with 1000 samples, 20 features, and 3 classes
    X, y = make_classification(
        n_samples=1000,
        n_features=20,
        n_informative=3,
        n_redundant=10,
        n_classes=3,
        random_state=42,
    )
    # Split the dataset into training and test sets
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=42
    )

    # Initialize the estimator
    est = estimator()

    # Use the hyperparameter search function provided by the BaseModel class to find the best parameters
    obj = BaseModel().optimize_by_tunesearchcv(
        kwargs={
            "fit_tune_kwargs": {"sample_weight": None},
            "tune_search_kwargs": {
                "estimator": est,
                "param_distributions": params,
                "scoring": "f1_micro",
                "verbose": 3,
                "n_jobs": -1,
                "cv": KFold(2),
            },
            "main_tune_kwargs": {},
        }
    )

    # Fit the model to the training data
    obj.fit(X_train, y_train)
    # Predict the labels for the test data
    y_pred = obj.predict(X_test)

    # Compute the F1 score of the model
    f1 = f1_score(y_test, y_pred, average="macro")
    print(f"f1_score is {f1}")


# Function for training and evaluating a regression model
def using_tune_regression(estimator, params):
    # Create a synthetic regression dataset with 1000 samples and 10 features
    X, y = make_regression(
        n_samples=1000, n_features=10, n_informative=5, n_targets=1, random_state=1
    )

    # Initialize the estimator
    est = estimator()

    # Use the hyperparameter search function provided by the BaseModel class to find the best parameters
    obj = BaseModel().optimize_by_tunesearchcv(
        kwargs={
            "fit_tune_kwargs": {},
            "tune_search_kwargs": {
                "estimator": est,
                "param_distributions": params,
                "scoring": "r2",
                "verbose": 3,
                "n_jobs": -1,
                "cv": KFold(2),
            },
            "main_tune_kwargs": {},
        }
    )

    # Fit the model to the data
    obj.fit(X, y)
    # Predict the targets for the data
    predictions = obj.predict(X)

    # Compute the R2 score of the model
    r2 = r2_score(y, predictions)
    print(f"r2_score is {r2}")


# Run the regression function using the MLPRegressor and the specified parameters
using_tune_regression(MLPRegressor, mlp_params_reg)

# Run the classification function using the LGBMClassifier and the specified parameters
using_tune_classification(LGBMClassifier, lgbm_params)


ModuleNotFoundError: No module named 'lohrasb.abstracts'